In [6]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import numpy as np
#id_left, text_left, id_left, text_right, label
#还要打乱
df0 = pd.read_excel('pipei_data.xlsx','Sheet1')

col_name=df0.columns.tolist()                   # 将数据框的列名全部提取出来存放在列表里
print(col_name)
col_name.insert(9,'host')
col_name.insert(10,'guest') # 在列索引为2的位置插入一列,列名为:city，刚插入时不会有值，整列都是NaN
col_name.insert(11,'label') 
df0=df0.reindex(columns=col_name)              # DataFrame.reindex() 对原行/列索引重新构建索引值

# main_product_host
# ami_product_guest
# df1 = pandas.DataFrame(df0, index, columns, dtype, copy)
list1 = []
for i in df0.index:
    list2 = []
    for j in range(1,5):
        m = df0.iloc[i,j]
        if pd.isna(m)==False:
            list2.append(m)
    list1.append(''.join(list2))
df0['host'] = pd.DataFrame(list1)
list3 = []
for i in df0.index:
    list4 = []
    for j in range(5,9):
        m = df0.iloc[i,j]
        if pd.isna(m)==False:
            list4.append(m)
    list3.append(''.join(list4))
df0['guest'] = pd.DataFrame(list3)
y = np.load('Y_pipei.npy')
df0['label'] = pd.DataFrame(y)
df1 = df0.loc[:,['host','guest','label']]
df1.to_excel('host_guest_label.xlsx','Sheet1')
df1 = shuffle(df1)

['Unnamed: 0', 'main_product', 'ambi_product', 'main_industry', 'ambi_industry', 'main_product.1', 'ambi_product.1', 'main_industry.1', 'ambi_industry.1']


In [4]:
df_origin = df1
df_train = df_origin.iloc[:6862,:]
df_train = df_train.reset_index(drop=True)
df_test = df_origin.iloc[6862:,:]
df_test = df_test.reset_index(drop=True)
df_train.to_excel('TRAIN.xlsx','Sheet1')
df_test.to_excel('TEST.xlsx','Sheet1')
df_test.head()

,host,guest,label
0,食品、饮料和烟草批发食品、饮料和烟草批发其他批发业,住院病人服务其他政府行政服务卫生卫生,0
1,"摩托车用发动机,机动车（汽车）零配件,摩托车零件、配件汽车、摩托车、燃料及零配件专门零售",机动车（汽车）零配件通用零部件制造,1
2,食品、饮料和烟草批发食品、饮料和烟草批发其他批发业其他批发业,"毛皮服装毛皮服装,油橄榄果,其他油料,干制水果及水果籽皮革、毛皮、羽毛及其制品和制鞋业农、林...",1
3,,"其他纺织制品针织或钩编床罩,针织或钩编相关床上制品,针织或钩编台布,针织或钩编相关餐桌用制品...",0
4,"皮革鞋靴,纺织面鞋,胶鞋,塑料鞋,木制鞋,舞蹈、戏剧用靴鞋,靴鞋零件、护腿及类似品,其他鞋纺...","旅游推广和访客信息服务,其他农业服务,园艺产品,其他林业服务,旅游星级饭店住宿服务,珠宝首饰...",1


,host,guest,label
3106,食用菌食用菌农业农业,管理咨询和管理服务管理咨询和管理服务国家机构国家机构,1
8437,"其他食品添加剂调味香料,其他储存和仓储服务,其他公路运输服务,其他铁路运输服务,铁路推或拖运...",家用洗衣机家用洗衣机电气机械和器材制造业电气机械和器材制造业,0
8565,针织Ｔ恤衫针织休闲衫纺织服装、服饰业纺织服装、服饰业,"塑料薄膜塑料管,塑料薄膜橡胶和塑料制品业橡胶和塑料制品业",0
7776,"纺织品、服装和鞋类批发纺织品、服装和鞋类批发纺织、服装及家庭用品批发,其他批发业纺织、服装及...",其他电工机械专用设备其他电工机械专用设备电子和电工机械专用设备制造电子和电工机械专用设备制造,0
2673,专用机械制造服务专用机械制造服务电子和电工机械专用设备制造汽车制造业,"车辆用电气音响信号装置,电气音响或视觉信号装置车辆用电气音响信号装置,汽车用音响设备,电气音...",1


In [9]:
import json
import codecs
import tensorflow as tf
from tqdm import tqdm


def sent2id(sent, vocab, max_size=30):
    sent = [vocab[c] for c in sent if c in vocab]
    sent = sent[:max_size] + [0]*(max_size - len(sent))
    return sent


def create_int_feature(values):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=values))


def convert_tfrecord(df,out_file,host_size,guest_size):
    vocab = json.load(codecs.open('char.json', "r", "utf-8"))
    #out_file = 'train.tfrecord'
    writer = tf.io.TFRecordWriter(out_file)
#     host_size = 200
#     guest_size = 200
    for i in df.index:
        host_s = df.iloc[i,0]
        guest_s = df.iloc[i,1]
        label = [df.iloc[i,2]]
        host_s = sent2id(host_s,vocab,host_size)
        guest_s = sent2id(guest_s,vocab,guest_size)
        feed_dict = {"host_char": create_int_feature(host_s),
                    "guest_char": create_int_feature(guest_s),
                    "label": create_int_feature(label)}
        example = tf.train.Example(features=tf.train.Features(feature=feed_dict))
        serialized = example.SerializeToString()
        writer.write(serialized)
    writer.close()

In [1]:
import tensorflow as tf

In [10]:
convert_tfrecord(df = df_train,out_file = 'train.tfrecord',host_size=50, guest_size=50)
convert_tfrecord(df = df_test,out_file = 'eval.tfrecord',host_size=50, guest_size=50)

In [8]:
df_excel = pd.read_excel('pipei_data.xlsx','Sheet1')
df_guest = df_excel['guest']
type(df_guest[0])

str

In [12]:
import json
vocab = json.load(codecs.open('char.json', "r", "utf-8"))
sent = "食品、饮料和烟草批发食品、饮料和烟草批发其他批发业其他批发业"
sent = [vocab.get(c, 1) for c in sent]

In [13]:
sent

[407,
 112,
 1,
 1075,
 550,
 19,
 1155,
 972,
 792,
 27,
 407,
 112,
 1,
 1075,
 550,
 19,
 1155,
 972,
 792,
 27,
 95,
 46,
 792,
 27,
 39,
 95,
 46,
 792,
 27,
 39]